# Module 2 Summative Lab

## Introduction

For today's section, we're going to work on a single big lab to apply everything we've learned in Mod 2!

## About This Lab

A quick note before getting started--this lab isn't like other labs you seen so far. This lab is meant to take ~8 hours to complete, so it's much longer and more challenging than the average labs you've seen so far. If you feel like this lab is challenging or that you might be struggling a bit, don't fret--that's by design! With everything we've learned about Web Scraping, APIs, and Databases, the best way to test our knowledge of it is to build something substantial! 

## The Project

In this lab, we're going to make use of everything we've learned about APIs, databases, and Object-Oriented Programming to **_Extract, Transform, and Load_** (or **_ETL_**, for short) some data from a SQL database into a MongoDB Database. 

You'll find a database containing information about soccer teams and the matches they've played in the file `database.sqlite`. For this project, our goal is to get the data we think is important from this SQL database, do some calculations and data transformation, and then store everything in a MongoDB database. 

Let's get into the specifics of this project.

### The Goal

Start by examining the data dictionary for the SQL database we'll be working with, which comes from this [kaggle page](https://www.kaggle.com/laudanum/footballdelphi).  Familiarize yourself with the tables it contains, and what each column means. We'll be using this database to get data on each soccer team, calculate some summary statistics, and then store each in a MongoDB database. 

Upon completion of this lab, each unique team in this dataset should have a record in the MongoDB instance containing the following information:

* The name of the team
* The total number of goals scored by the team during the 2011 season
* The total number of wins the team earned during the 2011 season
* A histogram visualization of the team's wins and losses for the 2011 season (store the visualization directly by assigning it to a variable)
* The team's win percentage on days where it was raining during games in the 2011 season. 

#### Getting the Weather Data

Note that for this last calculation, you'll need to figure out if it was raining or not during the game. The database itself does not contain this information, but it does contain the date on which the game was played. For this, you'll need to use the [DarkSky API](https://darksky.net/dev) to get the historical weather data for that day. Note that each game is played in a different location, and this information is not contained in our SQL database. However, the teams in this database are largely german, so go ahead and just use the weather in Berlin, Germany as a proxy for this information. If it was raining in Berlin on the day the game was played, count that as rain game--**_you do not need to try and figure out the actual weather at each game's location, because we don't have that information!_**

#### NOTE: The DarkSky API is limited to 1000 free API calls a day, so be sure to test your model on very small samples. Otherwise, you'll hit the rate limit!

## Project Architecture

Unlike previous labs, this lab is more open-ended, and will require you to make design decisions and plan out your strategy for building a system with this many working parts. However, **_using Object-Oriented Programming is a requirement for this project--you must create at least 2 separate, well-structured classes in your solution!_** Although it may seem easier to "just start coding", this is a classic beginner's mistake. Instead, think about separating out the different functionalities you'll need to reach your goal, and then build classes to handle each. For instance, at a minimum, you'll need to:

* Query the SQL database
* Calculate summary statistics
* Get the weather data from the DarkSky API
* Load the data into MongoDB

We **_strongly recommend_** you consider creating separate classes for handling at least some of these tasks.  Be sure to plan the inputs, outputs, and methods for each class before you begin coding! 

**_NOTE:_** We have provided some empty classes below. You are welcome to delete them and use a different architecture for this project if you so choose.  You do not have to use each of them, they are just there to give you an idea of what sorts of classes you may want to consider using.

### Rapid Prototyping and Refactoring

It's totally okay to try to get a task working without using OOP. For instance, when experimenting with the DarkSky API for getting historical weather data, it makes sense to just write the code in the cells and rapidly iterate until you get it all working. However, once you get it working, you're not done--you should then **_Refactor_** your code into functions or classes to make your code more modular, reusable, understandable, and maintainable! 

In short--do what you need to do to get each separate piece of functionality working, and then refactor it into a class after you've figured it out!

### Some Final Advice

You haven't built anything this big or complex thus far, so you may not yet fully realize how much trial and error goes into it. If your code keeps breaking, resist the urge to get frustrated, and just keep working. Software development is an iterative process!  No one writes perfect code that works the first time for something this involved. You're going to run into _a lot_ of small errors in this project, right up until the point where it just works, and then you're done! However, you can reduce these errors by planning out your code, and thinking about how all of the pieces fit together before you begin coding. Once you have some basic understanding of how it all will work, then you'll know what you need to build, and then all that is left is to build it!

In short:

* Plan ahead--you'll thank yourself later!
* Errors and broken code aren't bad, they're normal. 
* Keep working, and stay confident--you can do this!

Good luck--we look forward to seeing your completed project!

In [1]:
#Import necessary libraries
import sqlite3
import requests
import pymongo
import pandas as pd
import matplotlib.pyplot as plt
import json
import io
import base64

#Necessary classes

class Team_in_2011:
    
    def __init__(self, name=''):
        self.name = name
    
    def setname(self, name):
        self._name = name
    def getname(self):
        return self._name
    def delname(self):
        del self._name
    name = property(getname, setname, delname)
    
    def gethomedf(self):
        return self._homedf
    def sethomedf(self, df):
        self._homedf = df
    def delhomedf(self):
        del self._homedf
    homedf = property(gethomedf, sethomedf, delhomedf)
    
    def getawaydf(self):
        return self._awaydf
    def setawaydf(self, df):
        self._awaydf = df
    def delawaydf(self):
        del self._awaydf
    awaydf = property(getawaydf, setawaydf, delawaydf)
    
    def totalgoals(self):
        homegoals = self._homedf['FTHG'].sum()
        awaygoals = self._awaydf['FTAG'].sum()
        return int(homegoals + awaygoals)
    
    def totalwins(self):
        homewins = len(self._homedf[self._homedf['FTR']=='H'])
        awaywins = len(self._awaydf[self._awaydf['FTR']=='A'])
        return int(homewins + awaywins)
    
    def totalWDLs(self):
        wins = len(self._homedf[self._homedf['FTR']=='H']) + len(self._awaydf[self._awaydf['FTR']=='A'])
        draws = len(self._homedf[self._homedf['FTR']=='D']) + len(self._awaydf[self._awaydf['FTR']=='D'])
        losses = len(self._homedf[self._homedf['FTR']=='A']) + len(self._awaydf[self._awaydf['FTR']=='H'])
        return {'wins': wins, 'draws': draws, 'losses': losses}
    
    def barWDLs(self):
        wdldict = self.totalWDLs()
        my_bytesIObytes = io.BytesIO()
        fig = plt.figure()
        plt.bar(['Wins', 'Draws','Losses'],[wdldict['wins'],wdldict['draws'],wdldict['losses']],figure=fig)
        plt.title(self._name+' 2011 Match Results')
        plt.savefig(my_bytesIObytes, format='png')
        my_bytesIObytes.seek(0)
        my_base64_pngData = base64.b64encode(my_bytesIObytes.read())
        plt.close()
        return my_base64_pngData

    def rainywinpercentage(self):
        rainywins = len(self._homedf[(self._homedf['FTR']=='H')&(self._homedf['Rain']=='Y')]) + len(self._awaydf[(self._awaydf['FTR']=='A')&(self._awaydf['Rain']=='Y')])
        rainydraws = len(self._homedf[(self._homedf['FTR']=='D')&(self._homedf['Rain']=='Y')]) + len(self._awaydf[(self._awaydf['FTR']=='D')&(self._awaydf['Rain']=='Y')])
        rainylosses = len(self._homedf[(self._homedf['FTR']=='A')&(self._homedf['Rain']=='Y')]) + len(self._awaydf[(self._awaydf['FTR']=='H')&(self._awaydf['Rain']=='Y')])
        rainytotal = rainywins + rainydraws + rainylosses
        return float(round((rainywins/rainytotal)*100,2))
    
#####################################################################

class Weather_Getter:
  
    _baseurl = 'https://api.darksky.net/forecast/'
    _berlinlatlong = '52.5200,13.4050'
    _londonlatlong = '51.5074,0.1278'
    
    with open('/Users/johnnguyen/.secret/darksky_api.json') as f:
        keys = json.load(f)
        _apikey = keys['api_key']
        
    @classmethod
    def getWeatherList(cls, matchesdf):
        weatherList = []
        unix_dates = list((pd.to_numeric(pd.to_datetime(matchesdf['Date']))/10**9).astype('int'))
        
        for (i, div) in enumerate(matchesdf['Div']):

            if div == 'D1' or div == 'D2':
                latlong = cls._berlinlatlong
            else:
                latlong = cls._londonlatlong

            date = unix_dates[i]
            url = cls._baseurl + cls._apikey + '/' + latlong + ',' + str(date)
            resp = requests.get(url)
            #print(resp.status_code)
            resp_dict = json.loads(resp.text)
            resp_dict_daily = resp_dict['daily']['data'][0]

            if 'icon' in resp_dict_daily.keys() and resp_dict_daily['icon'] == 'rain':
                weatherList.append('Y')
            else:
                weatherList.append('N')

        return weatherList
    
#####################################################################

class Mongo_Handler:
    
    def __init__(self, db_address, db_name, collection_name):
        self._myclient = pymongo.MongoClient(db_address)
        self._mydb = self._myclient[db_name]
        self._mycollection = self._mydb[collection_name]
        
    def insertone(self, record):
        self._mycollection.insert_one(record)
    
    def insertmany(self, records):
        self._mycollection.insert_many(records)
        
    def printrecords(self):
        query = self._mycollection.find({})
        for x in query:
            print(x)
    
    

In [2]:
#Establish SQL connection and cursor
connection = sqlite3.connect('database.sqlite')
cursor = connection.cursor()

#Connect to MongoDB
mongo = Mongo_Handler('mongodb://127.0.0.1:27017/', 'football_results', 'team_results_2011')



In [3]:
#Previewing Matches in 2011
results = cursor.execute('''
                SELECT * FROM Matches
                WHERE Season == 2011;
                ''').fetchall()

matches_2011_df = pd.DataFrame(results)
matches_2011_df.columns = [i[0] for i in cursor.description]
print(len(matches_2011_df))
matches_2011_df.head()



992


,Match_ID,Div,Season,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR
0,1092,D1,2011,2012-03-31,Nurnberg,Bayern Munich,0,1,A
1,1093,D1,2011,2011-12-11,Stuttgart,Bayern Munich,1,2,A
2,1094,D1,2011,2011-08-13,Wolfsburg,Bayern Munich,0,1,A
3,1095,D1,2011,2011-11-27,Mainz,Bayern Munich,3,2,H
4,1096,D1,2011,2012-02-18,Freiburg,Bayern Munich,0,0,D


In [4]:
#Unique teams in matches_2011_df

unique_teams_2011 = sorted(list(set(pd.concat((matches_2011_df['AwayTeam'],matches_2011_df['HomeTeam'])))))
print(len(unique_teams_2011))
print(unique_teams_2011)


56
['Aachen', 'Arsenal', 'Aston Villa', 'Augsburg', 'Bayern Munich', 'Blackburn', 'Bochum', 'Bolton', 'Braunschweig', 'Chelsea', 'Cottbus', 'Dortmund', 'Dresden', 'Duisburg', 'Ein Frankfurt', 'Erzgebirge Aue', 'Everton', 'FC Koln', 'Fortuna Dusseldorf', 'Frankfurt FSV', 'Freiburg', 'Fulham', 'Greuther Furth', 'Hamburg', 'Hannover', 'Hansa Rostock', 'Hertha', 'Hoffenheim', 'Ingolstadt', 'Kaiserslautern', 'Karlsruhe', 'Leverkusen', 'Liverpool', "M'gladbach", 'Mainz', 'Man City', 'Man United', 'Munich 1860', 'Newcastle', 'Norwich', 'Nurnberg', 'Paderborn', 'QPR', 'Schalke 04', 'St Pauli', 'Stoke', 'Stuttgart', 'Sunderland', 'Swansea', 'Tottenham', 'Union Berlin', 'Werder Bremen', 'West Brom', 'Wigan', 'Wolfsburg', 'Wolves']


In [8]:
mongo._mycollection.delete_many({})
mongo.printrecords()

In [9]:
#Master loop
for team in unique_teams_2011[:20]:
    print(team)
    
    #Filter home and away results for specific team
    home_matches_df = matches_2011_df[matches_2011_df['HomeTeam'] == team]
#    print(home_matches_df.head())
    away_matches_df = matches_2011_df[matches_2011_df['AwayTeam'] == team]
#    print(away_matches_df.head())
    print('Home and away dataframes created.')
    
    #DarkSky API call to add match rain information to both home and away dfs
    home_matches_df['Rain'] = Weather_Getter.getWeatherList(home_matches_df)
#    print(home_matches_df)
    away_matches_df['Rain'] = Weather_Getter.getWeatherList(away_matches_df)
#    print(away_matches_df)
    print('Rainy match information added to dataframes.')
    
    #Create Team objects to hold information of interest
    team_object = Team_in_2011(team)
    team_object.homedf = home_matches_df
    team_object.awaydf = away_matches_df
    print('Team object created.')

    #Create record to be inserted into MongoDB
    team_record = {'name': team_object.name, 'total_goals': team_object.totalgoals(), 'total_wins': team_object.totalwins(), 'results_graph': team_object.barWDLs(), 'rainy_win_percentage': team_object.rainywinpercentage()}
    results = mongo.insertone(team_record)
    print('Record added to database.')


Aachen
Home and away dataframes created.


/Users/johnnguyen/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]
/Users/johnnguyen/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


Rainy match information added to dataframes.
Team object created.
Record added to database.
Arsenal
Home and away dataframes created.
Rainy match information added to dataframes.
Team object created.
Record added to database.
Aston Villa
Home and away dataframes created.
Rainy match information added to dataframes.
Team object created.
Record added to database.
Augsburg
Home and away dataframes created.
Rainy match information added to dataframes.
Team object created.
Record added to database.
Bayern Munich
Home and away dataframes created.
Rainy match information added to dataframes.
Team object created.
Record added to database.
Blackburn
Home and away dataframes created.
Rainy match information added to dataframes.
Team object created.
Record added to database.
Bochum
Home and away dataframes created.
Rainy match information added to dataframes.
Team object created.
Record added to database.
Bolton
Home and away dataframes created.
Rainy match information added to dataframes.
Team o

KeyError: 'daily'

In [13]:
for x in mongo._mycollection.find({'name': 'Bayern Munich'}):
    print(x)


{'_id': ObjectId('5deb549feff7505bdd37ad2f'), 'name': 'Bayern Munich', 'total_goals': 77, 'total_wins': 23, 'results_graph': b'iVBORw0KGgoAAAANSUhEUgAAAbAAAAEgCAYAAADVKCZpAAAABHNCSVQICAgIfAhkiAAAAAlwSFlzAAALEgAACxIB0t1+/AAAADl0RVh0U29mdHdhcmUAbWF0cGxvdGxpYiB2ZXJzaW9uIDMuMC4yLCBodHRwOi8vbWF0cGxvdGxpYi5vcmcvOIA7rQAAHYpJREFUeJzt3XlYlXX+//EXAikKCMgiikmooSMqY6ShoRmBJSOWJqamoBbaaM2VTuk1jY1Z41iNZY05hpliuVXXlZZbGmmTezjhksugJhESQmKJC7J8fn/463xjpLRGOedjz8d1eV3cyznnfc4NPLlvTuRmjDECAMAy9Zw9AAAAvwQBAwBYiYABAKxEwAAAViJgAAArETAAgJUIGADASgQMAGAlAgYAsBIBAwBYiYABAKxEwAAAViJgAAArETAAgJUIGADASgQMAGAlAgYAsBIBAwBYiYDhV2XMmDF6+umnL7nfbbfdptdee60OJrKfm5ubDh065OwxflJ4eLg+/PBDZ4+BK4yAXWPCw8Pl5eUlb29v+fv7KykpSfn5+c4e65KOHj0qNzc3de7cucb6kpISXXfddQoPD78ijzNnzhxNnjz5itzX97Zt26aEhAQFBAQoKChIAwcOVGFhoWO7MUYTJ05UkyZN1KRJEz3++OMyxji2p6enKzIyUvXq1dOCBQtq3PfevXvVu3dvBQYGys3N7ZKzuLm5KSQkRJWVlY51lZWVCg4OvqzbS9LGjRsVFhZ2Wfv+XFOmTJGnp6e8vb3l5+enbt26aevWrVflsX5qhvvvv79OHxNXBwG7Br3//vsqKytTYWGhQkJC9PDDD9f5DFVVVb/odqdPn9bevXsdy

# Summary

In this lab, we dug deep and used everything we've learned so far about python programming, databases, HTTP requests and API calls to ETL data from a SQL database into a MongoDB instance!